<a href="https://colab.research.google.com/github/arbabcodess/pratice/blob/main/yolov11_densenet_dem_ohrc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Collab Notebook Test - 08 <br>
Date : 17th May 2025 <br>
Status : No Errors in loading the modified model with densenet169.

## **Checking the GPU-config.**

In [ ]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


## **Mounting the .zip script file in the Drive and loading the dataset and the modified .yaml cfg file for 'densenet169' in the runtime environment.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

drive_file_name = "dataset_yolov11_dem_ohrc"
dataset_name = "CH2.v2i.yolov11"

file_location_on_drive = "/content/drive/MyDrive/" + drive_file_name + ".zip"
!cp "{file_location_on_drive}" /content/

zip_file_location_runtime = "/content/" + drive_file_name + ".zip"
!unzip -q "{zip_file_location_runtime}" -d /content/

dataset_zip_file_location_runtime = "/content/" + dataset_name + ".zip"
dataset_folder_name = "dataset"
dataset_folder_location = "/content/" + dataset_folder_name
!unzip -q "{dataset_zip_file_location_runtime}" -d "{dataset_folder_location}"

dataset_yaml_file = dataset_folder_name + "/data.yaml"
!cat "{dataset_yaml_file}"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
replace /content/__MACOSX/._CH2.v2i.yolov11.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
train: ../train/images
val: ../valid/images
test: ../test/images

nc: 1
names: ['crater']

roboflow:
  workspace: depo-mtj0m
  project: ch2-qlnt2
  version: 2
  license: MIT
  url: https://universe.roboflow.com/depo-mtj0m/ch2-qlnt2/dataset/2

## **Cloning the {modified} Ultralytics GitHub repo.**

In [ ]:
!git clone https://github.com/ultralytics/ultralytics
%cd ultralytics

!git fetch origin pull/19609/head:pr-19609
!git checkout pr-19609

Cloning into 'ultralytics'...
remote: Enumerating objects: 59708, done.
remote: Counting objects: 100% (174/174), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 59708 (delta 118), reused 70 (delta 55), pack-reused 59534 (from 2)
Receiving objects: 100% (59708/59708), 32.51 MiB | 21.13 MiB/s, done.
Resolving deltas: 100% (44380/44380), done.
/content/ultralytics
remote: Enumerating objects: 342, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 342 (delta 202), reused 202 (delta 202), pack-reused 138 (from 2)
Receiving objects: 100% (342/342), 96.00 KiB | 3.20 MiB/s, done.
Resolving deltas: 100% (235/235), completed with 105 local objects.
From https://github.com/ultralytics/ultralytics
 * [new ref]           refs/pull/19609/head -> pr-19609
Switched to branch 'pr-19609'


## **List of all the models present in the 'Timm' module.**

In [ ]:
import timm
models = timm.list_models()
for model in models:
    print(model)

aimv2_1b_patch14_224
aimv2_1b_patch14_336
aimv2_1b_patch14_448
aimv2_3b_patch14_224
aimv2_3b_patch14_336
aimv2_3b_patch14_448
aimv2_huge_patch14_224
aimv2_huge_patch14_336
aimv2_huge_patch14_448
aimv2_large_patch14_224
aimv2_large_patch14_336
aimv2_large_patch14_448
bat_resnext26ts
beit_base_patch16_224
beit_base_patch16_384
beit_large_patch16_224
beit_large_patch16_384
beit_large_patch16_512
beitv2_base_patch16_224
beitv2_large_patch16_224
botnet26t_256
botnet50ts_256
caformer_b36
caformer_m36
caformer_s18
caformer_s36
cait_m36_384
cait_m48_448
cait_s24_224
cait_s24_384
cait_s36_384
cait_xs24_384
cait_xxs24_224
cait_xxs24_384
cait_xxs36_224
cait_xxs36_384
coat_lite_medium
coat_lite_medium_384
coat_lite_mini
coat_lite_small
coat_lite_tiny
coat_mini
coat_small
coat_tiny
coatnet_0_224
coatnet_0_rw_224
coatnet_1_224
coatnet_1_rw_224
coatnet_2_224
coatnet_2_rw_224
coatnet_3_224
coatnet_3_rw_224
coatnet_4_224
coatnet_5_224
coatnet_bn_0_rw_224
coatnet_nano_cc_224
coatnet_nano_rw_224
coatnet_

## **Using 'densenet169'**

*   Printing the feature-map tensor for the 'densenet169' backbone.
*   {used for the custom yolo11.yaml file}

In [ ]:
from ultralytics.nn.modules.block import Timm
import torch

# The Timm constructor has the following arguments (based on Ultralytics' wrapper):
# Timm(model_name, pretrained=True, features_only=True, out_indices=0, export=False)
# .eval() puts the model in inference mode.
mods = Timm('densenet169', True, True, 0, True).eval()

# A single (batch size = 1) RGB image of shape 640×640 is simulated.
img = torch.randn(1, 3, 640, 640)

'''
  img = torch.randn(1, 3, H, W)
  out = model(img)

  above will run as long as:
  H and W are divisible enough for the downsampling layers (like pooling and strides).
  The input isn’t too small, which could cause the spatial dimensions to collapse after multiple downsampling stages.
'''

out = mods(img)
for i in range(len(out)):
    print(i, out[i].shape)

# Each out[i] is a tensor representing a feature map from a specific stage in the backbone.


# changing the cfg yolo11.yaml file
!cp /content/yolo11.yaml /content/ultralytics/ultralytics/cfg/models/11/yolo11.yaml

0 torch.Size([1, 64, 320, 320])
1 torch.Size([1, 256, 160, 160])
2 torch.Size([1, 512, 80, 80])
3 torch.Size([1, 1280, 40, 40])
4 torch.Size([1, 1664, 20, 20])


## **Model Loading...**

In [ ]:
from ultralytics import YOLO

model = YOLO("yolo11.yaml")

## **Model Training...**

In [ ]:
e = int(input("Enter the no. of epochs."))
res = model.train(data="/content/dataset/data.yaml", epochs=e)

Enter the no. of epochs.1
New https://pypi.org/project/ultralytics/8.3.137 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.133 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opse

train: Scanning /content/dataset/train/labels.cache... 20 images, 0 backgrounds, 0 corrupt: 100%|██████████| 20/20 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1295.1±969.7 MB/s, size: 60.4 KB)



val: Scanning /content/dataset/valid/labels.cache... 3 images, 0 backgrounds, 0 corrupt: 100%|██████████| 3/3 [00:00<?, ?it/s]


Plotting labels to /content/ultralytics/runs/detect/train5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 217 weight(decay=0.0), 223 weight(decay=0.0005), 223 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to /content/ultralytics/runs/detect/train5
Starting training for 1 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


  0%|          | 0/2 [00:00<?, ?it/s]